# Deep Learning

In [ ]:
# Boilerplate to chdir to project root and add projects sources to python path
import os
import sys
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
    sys.path.append(os.path.abspath('.'))

# Prepare the dataset
import biosignals.prepare as bp
if not bp.has_prepared('rand'):
    bp.prepare_rand()

In [ ]:
# Boilerplate to chdir to project root and add projects sources to python path
# ---DELETE---
import os
import sys
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
    sys.path.append(os.path.abspath('.'))

import biosignals.deep_models as bdeep
# ---DELETE---

bdeep.test_models()

In [ ]:
import biosignals.deep_models as bdeep
# Test deep learning models (LSTM, GRU) so far
bdeep.test_models()

# The rest of this notebook is redundant.

In [ ]:
import biosignals.prepare as bp
import biosignals.evaluation as be
import biosignals.models as bm
from biosignals.split import Role
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU, Activation
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA

In [ ]:
prepared = bp.read_prepared("rand")
print(prepared)

In [ ]:
data = prepared[Role.TRAIN][0].load()
data_val = prepared[Role.VALIDATE][0].load()

In [ ]:
# Eric's Random Forest
bm.test_models()

In [ ]:
from biosignals.models import *
# Hacky way to obtain the clustered data by abusing Eric's work...
data = bp.read_clusters()
model = bm.SkModel(RandomForestClassifier, {}, Strategy.MULTI)
lds = bp.read_prepared('rand')
data = model._load_all(lds[bs.Role.TRAIN], RandomState(42))
data_val = model._load_all(lds[bs.Role.VALIDATE], RandomState(42))
data_test = model._load_all(lds[bs.Role.TEST], RandomState(42))

In [ ]:
print(data[0].shape, data[1].shape, data_val[0].shape, data_val[1].shape, data_test[0].shape, data_test[1].shape)

In [ ]:
train_data = tf.convert_to_tensor(data[0], dtype=tf.float64)
train_labels = tf.convert_to_tensor(data[1], dtype=tf.int32)
print(train_labels.shape, train_data.shape)

val_data = tf.convert_to_tensor(data_val[0], dtype=tf.float64)
val_labels = tf.convert_to_tensor(data_val[1], dtype=tf.int32)

test_data = tf.convert_to_tensor(data_test[0], dtype=tf.float64)
test_labels = tf.convert_to_tensor(data_test[1], dtype=tf.int32)

In [ ]:
#standard scalar
scaler = StandardScaler()
scaler.fit(train_data)
train_data = scaler.transform(train_data)
val_data = scaler.transform(val_data)
test_data = scaler.transform(test_data)

#PCA
n_components = 64
pca = PCA(n_components=n_components)
pca.fit(train_data)
train_data_pca = pca.transform(train_data)
val_data_pca = pca.transform(val_data)
test_data_pca = pca.transform(test_data)


# Expand dimensions
train_data_pca = tf.expand_dims(train_data_pca, axis=1)
val_data_pca = tf.expand_dims(val_data_pca, axis=1)
test_data_pca = tf.expand_dims(test_data_pca, axis=1)

train_data_nopca = tf.expand_dims(train_data, axis=1)
val_data_nopca = tf.expand_dims(val_data, axis=1)
test_data_nopca = tf.expand_dims(test_data, axis=1)

print(train_data_pca.shape, val_data_pca.shape, test_data_pca.shape)
print(train_data_nopca.shape, val_data_nopca.shape, test_data_nopca.shape)

In [ ]:
# GRU NO PCA
model_GRU_nopca = bdeep.GRU_Model(train_data_nopca, train_labels, num_epochs=30, batch_size=64)
be.evaluate_model(model_GRU_nopca.predict(val_data_nopca), val_labels)

In [ ]:
# GRU WITH PCA
model_GRU_pca = bdeep.GRU_Model(train_data_pca, train_labels, num_epochs=30, batch_size=64)
be.evaluate_model(model_GRU_pca.predict(val_data_pca), val_labels)

In [ ]:
# LSTM NO PCA
model_LSTM_nopca = bdeep.LSTM_Model(train_data_nopca, train_labels, num_epochs=30, batch_size=64)
be.evaluate_model(model_LSTM_nopca.predict(val_data_nopca), val_labels)

In [ ]:
# LSTM WITH PCA
model_LSTM_pca = bdeep.LSTM_Model(train_data_pca, train_labels, num_epochs=30, batch_size=64)
be.evaluate_model(model_LSTM_pca.predict(val_data_pca), val_labels)

In [ ]:
# Test evaluation
print("GRU - no PCA")
be.evaluate_model(model_GRU_nopca.predict(test_data_nopca), test_labels)
print("GRU - with PCA")
be.evaluate_model(model_GRU_pca.predict(test_data_pca), test_labels)

print("LSTM - no PCA")
be.evaluate_model(model_LSTM_nopca.predict(test_data_nopca), test_labels)
print("LSTM - with PCA")
be.evaluate_model(model_LSTM_pca.predict(test_data_pca), test_labels)

In [ ]:
# Training evaluation (to see if we overfit)
print("GRU - no PCA")
be.evaluate_model(model_GRU_nopca.predict(train_data_nopca), train_labels)
print("GRU - PCA")
be.evaluate_model(model_GRU_pca.predict(train_data_pca), train_labels)
print("LSTM - no PCA")
be.evaluate_model(model_LSTM_nopca.predict(train_data_nopca), train_labels)
print("LSTM - PCA")
be.evaluate_model(model_LSTM_pca.predict(train_data_pca), train_labels)